In [3]:
# -*- coding: utf-8 -*-
# 단일 모델 학습: RTMDet(우선) 기반
# COCO 기본 evaluator 경로를 완전히 덮어써서 instances_val2017.json 오류를 방지

# 1) split 생성
assert TRAIN_JSON_FULL.exists(), f"train.json 없음: {TRAIN_JSON_FULL}"

split_dir = FULL_DATA_ROOT / "splits_single"
split_json = make_train_val_split(
    src_json=TRAIN_JSON_FULL,
    out_dir=split_dir,
    train_ratio=TRAIN_RATIO,
    seed=RANDOM_SEED
)

print("train split:", split_json["train"])
print("val split  :", split_json["val"])


# 2) base cfg 선택
register_all_modules(init_default_scope=True)
base_cfg_path = pick_best_single_base_cfg()

# 3) Config 로드
cfg = Config.fromfile(str(base_cfg_path))
cfg.default_scope = "mmdet"

# 4) 데이터 루트 고정
cfg.data_root = str(FULL_DATA_ROOT)

# 5) train/val dataloader dataset 교체
for key, ann_path in [
    ("train_dataloader", split_json["train"]),
    ("val_dataloader", split_json["val"]),
]:
    loader = cfg[key]
    ds_cfg = loader["dataset"] if "dataset" in loader else loader

    # 클래스 메타정보
    ds_cfg.metainfo = dict(classes=CLASSES)
    ds_cfg.data_root = str(FULL_DATA_ROOT)

    # ann_file은 절대경로로 고정
    ds_cfg.ann_file = str(ann_path)

    # train.json 내부 file_name이 "train/xxx.jpg" 형태라고 가정
    # 따라서 img prefix는 빈 문자열 유지
    ds_cfg.data_prefix = dict(img="")

# 6) 스케일 및 클래스 수 고정
# pipeline 구조가 config마다 다를 수 있어 안전 처리
if "dataset" in cfg.train_dataloader and hasattr(cfg.train_dataloader.dataset, "pipeline"):
    set_img_scale(cfg.train_dataloader.dataset.pipeline, IMAGE_SCALE)
if "dataset" in cfg.val_dataloader and hasattr(cfg.val_dataloader.dataset, "pipeline"):
    set_img_scale(cfg.val_dataloader.dataset.pipeline, IMAGE_SCALE)

set_num_classes(cfg.model, len(CLASSES))


# 7) evaluator 완전 덮어쓰기 (핵심)
# 여기서 COCO 기본 경로를 반드시 제거해야 함
cfg.val_evaluator = dict(
    type="CocoMetric",
    ann_file=str(split_json["val"]),
    metric="bbox",
    iou_thrs=[0.5],
    format_only=False,
)

# test()를 val mAP 계산에 쓰기 위해 test_dataloader/evaluator도 동일하게 맞춤
cfg.test_dataloader = cfg.val_dataloader
cfg.test_evaluator = dict(
    type="CocoMetric",
    ann_file=str(split_json["val"]),
    metric="bbox",
    iou_thrs=[0.5],
    format_only=False,
)

# 8) 배치/워커
cfg.train_dataloader.batch_size = 2
cfg.train_dataloader.num_workers = 4
cfg.val_dataloader.batch_size = 1
cfg.val_dataloader.num_workers = 2
cfg.test_dataloader.batch_size = 1
cfg.test_dataloader.num_workers = 2


# 9) 학습 epoch 및 검증 주기
# base가 300e 같은 장기 스케줄일 수 있어
# 단일 GPU 실습 환경에서는 현실적 길이로 조정
cfg.train_cfg.max_epochs = 12
cfg.train_cfg.val_interval = 1
cfg.val_cfg = dict(type="ValLoop")

# 10) 러닝레이트 보정
# base lr가 큰 경우가 많으므로 단일 GPU/bs=2 기준 보수적 스케일링
if hasattr(cfg, "optim_wrapper") and "optimizer" in cfg.optim_wrapper:
    base_lr = cfg.optim_wrapper.optimizer.get("lr", 0.02)
    cfg.optim_wrapper.optimizer["lr"] = base_lr * 2 / 16

# 11) 체크포인트/로그
cfg.default_hooks["checkpoint"]["interval"] = 1
cfg.default_hooks["checkpoint"]["max_keep_ckpts"] = 2
cfg.default_hooks["logger"]["interval"] = 50

# 12) work_dir/시드 고정
EXP_NAME = "rtmdet_1024_single"
cfg.work_dir = str(WORK_DIR_ROOT / EXP_NAME)
cfg.randomness = dict(seed=RANDOM_SEED, deterministic=False)

print("Final work_dir:", cfg.work_dir)

# 13) 학습 실행
runner = Runner.from_cfg(cfg)
runner.train()

# 14) val mAP50 확인
metrics = runner.test()
print("val metrics:", metrics)
print("학습 완료")


train split: /data/ephemeral/home/model/dataset/splits_single/train_split.json
val split  : /data/ephemeral/home/model/dataset/splits_single/val_split.json
Selected base cfg: /data/ephemeral/home/model/baseline/mmdetection/configs/rtmdet/rtmdet_l_8xb32-300e_coco.py
Final work_dir: /data/ephemeral/home/model/work_dirs_single/rtmdet_1024_single
12/07 10:13:58 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 42
    GPU 0: Tesla V100-SXM2-32GB
    CUDA_HOME: None
    GCC: n/a
    PyTorch: 2.1.2+cu121
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde

/bin/sh: 1: gcc: not found


12/07 10:13:59 - mmengine - INFO - Config:
auto_scale_lr = dict(base_batch_size=16, enable=False)
backend_args = None
base_lr = 0.004
custom_hooks = [
    dict(
        ema_type='ExpMomentumEMA',
        momentum=0.0002,
        priority=49,
        type='EMAHook',
        update_buffers=True),
    dict(
        switch_epoch=280,
        switch_pipeline=[
            dict(backend_args=None, type='LoadImageFromFile'),
            dict(type='LoadAnnotations', with_bbox=True),
            dict(
                keep_ratio=True,
                ratio_range=(
                    0.1,
                    2.0,
                ),
                scale=(
                    640,
                    640,
                ),
                type='RandomResize'),
            dict(crop_size=(
                640,
                640,
            ), type='RandomCrop'),
            dict(type='YOLOXHSVRandomAug'),
            dict(prob=0.5, type='RandomFlip'),
            dict(
                pad_val=d

/data/ephemeral/home/py310/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


12/07 10:14:26 - mmengine - INFO - Epoch(train)  [1][  50/2197]  base_lr: 2.4529e-05 lr: 2.4529e-05  eta: 2:55:53  time: 0.4011  data_time: 0.0123  memory: 3551  loss: 2.0806  loss_cls: 0.8156  loss_bbox: 1.2650
12/07 10:14:43 - mmengine - INFO - Epoch(train)  [1][ 100/2197]  base_lr: 4.9554e-05 lr: 4.9554e-05  eta: 2:44:42  time: 0.3515  data_time: 0.0042  memory: 3587  loss: 2.0980  loss_cls: 0.8651  loss_bbox: 1.2329
12/07 10:15:01 - mmengine - INFO - Epoch(train)  [1][ 150/2197]  base_lr: 7.4579e-05 lr: 7.4579e-05  eta: 2:40:38  time: 0.3506  data_time: 0.0039  memory: 3540  loss: 2.0614  loss_cls: 0.9028  loss_bbox: 1.1585
12/07 10:15:19 - mmengine - INFO - Epoch(train)  [1][ 200/2197]  base_lr: 9.9604e-05 lr: 9.9604e-05  eta: 2:38:55  time: 0.3547  data_time: 0.0042  memory: 3549  loss: 2.0297  loss_cls: 0.9083  loss_bbox: 1.1213
12/07 10:15:36 - mmengine - INFO - Epoch(train)  [1][ 250/2197]  base_lr: 1.2463e-04 lr: 1.2463e-04  eta: 2:37:16  time: 0.3490  data_time: 0.0042  memo

In [2]:
# -*- coding: utf-8 -*-
# ipynb 공통 유틸: 경로, split 생성, config 안전 패치

import json
import random
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import pandas as pd

from mmengine.config import Config
from mmengine.runner import Runner
from mmdet.utils import register_all_modules


# -----------------------
# 1) 고정 경로 (네 환경 기준)
# -----------------------

# mmdetection 루트 (configs 탐색용)
MMD_ROOT = Path("/data/ephemeral/home/model/baseline/mmdetection")

# 데이터 루트
FULL_DATA_ROOT = Path("/data/ephemeral/home/model/dataset")
TRAIN_JSON_FULL = FULL_DATA_ROOT / "train.json"
TEST_JSON_FULL  = FULL_DATA_ROOT / "test.json"

# 샘플 제출 폴더
SAMPLE_SUB_DIR = Path("/data/ephemeral/home/model/sample_submission")

# 단일 모델 실험 산출물 저장 위치
WORK_DIR_ROOT = Path("/data/ephemeral/home/model/work_dirs_single")
WORK_DIR_ROOT.mkdir(parents=True, exist_ok=True)

# 이미지 스케일 고정
IMAGE_SCALE = (1024, 1024)

# 클래스 10개 (대회 공지 기준 순서)
CLASSES = (
    "General trash",
    "Paper",
    "Paper pack",
    "Metal",
    "Glass",
    "Plastic",
    "Styrofoam",
    "Plastic bag",
    "Battery",
    "Clothing",
)

# split 설정
RANDOM_SEED = 42
TRAIN_RATIO = 0.9   # 0.8/0.2보다 학습 데이터를 늘려 단일 모델 성능 안정성을 노림


print("MMD_ROOT:", MMD_ROOT)
print("FULL_DATA_ROOT:", FULL_DATA_ROOT)
print("TRAIN_JSON_FULL:", TRAIN_JSON_FULL)
print("TEST_JSON_FULL :", TEST_JSON_FULL)
print("SAMPLE_SUB_DIR :", SAMPLE_SUB_DIR)
print("WORK_DIR_ROOT  :", WORK_DIR_ROOT)


# -----------------------
# 2) COCO json 로드/저장
# -----------------------

def load_coco(json_path: Path) -> Dict:
    with open(json_path, "r") as f:
        return json.load(f)

def save_coco(data: Dict, json_path: Path) -> None:
    json_path.parent.mkdir(parents=True, exist_ok=True)
    with open(json_path, "w") as f:
        json.dump(data, f)


# -----------------------
# 3) train/val split 생성
# -----------------------

def make_train_val_split(
    src_json: Path,
    out_dir: Path,
    train_ratio: float = 0.9,
    seed: int = 42
) -> Dict[str, Path]:
    """
    이미지 id 기준 랜덤 분할.
    detection에서 가장 기본적이고 안전한 방식.
    """
    random.seed(seed)

    data = load_coco(src_json)
    images = data["images"]
    anns = data["annotations"]

    img_ids = [img["id"] for img in images]
    random.shuffle(img_ids)

    split_idx = int(len(img_ids) * train_ratio)
    train_ids = set(img_ids[:split_idx])
    val_ids   = set(img_ids[split_idx:])

    def _filter(ids: set):
        # ids에 해당하는 image만 남김
        imgs = [img for img in images if img["id"] in ids]
        img_set = {img["id"] for img in imgs}
        # 해당 이미지에 매칭되는 annotation만 남김
        filtered_anns = [ann for ann in anns if ann["image_id"] in img_set]
        return {**data, "images": imgs, "annotations": filtered_anns}

    out_dir.mkdir(parents=True, exist_ok=True)
    train_json = out_dir / "train_split.json"
    val_json   = out_dir / "val_split.json"

    save_coco(_filter(train_ids), train_json)
    save_coco(_filter(val_ids), val_json)

    return {"train": train_json, "val": val_json}


# -----------------------
# 4) pipeline 스케일 고정
# -----------------------

def set_img_scale(pipeline, scale):
    """
    Resize / RandomResize / RandomChoiceResize 등을 1024로 통일.
    """
    for t in pipeline:
        if isinstance(t, list):
            set_img_scale(t, scale)
            continue
        if not isinstance(t, dict):
            continue

        if t.get("type") in ("Resize", "RandomResize", "RandomChoiceResize"):
            if "scale" in t:
                t["scale"] = scale
            if "img_scale" in t:
                t["img_scale"] = scale
            if "scales" in t:
                t["scales"] = [scale]

        if "transforms" in t:
            set_img_scale(t["transforms"], scale)


# -----------------------
# 5) num_classes 재귀 고정
# -----------------------

def set_num_classes(model_cfg, num_classes: int):
    """
    다양한 head 구조에서 num_classes를 재귀적으로 10으로 맞춤.
    """
    if isinstance(model_cfg, dict):
        if "num_classes" in model_cfg:
            model_cfg["num_classes"] = num_classes
        for v in model_cfg.values():
            set_num_classes(v, num_classes)
    elif isinstance(model_cfg, list):
        for v in model_cfg:
            set_num_classes(v, num_classes)


# -----------------------
# 6) 샘플 csv 자동 선택
# -----------------------

def pick_sample_csv(sample_dir: Path) -> Path:
    """
    - 이름에 mmdetection 포함된 샘플을 우선
    - 없으면 첫 번째 csv
    """
    csvs = sorted(sample_dir.glob("*.csv"))
    if not csvs:
        raise FileNotFoundError(f"샘플 제출 csv가 없습니다: {sample_dir}")

    for c in csvs:
        if "mmdetection" in c.name.lower():
            return c

    return csvs[0]


# -----------------------
# 7) base config 자동 탐색
# -----------------------

def pick_best_single_base_cfg() -> Path:
    """
    앙상블에 없던 단일 후보로 RTMDet 계열을 우선 시도.
    환경에 따라 config 파일명이 다를 수 있어
    여러 후보 중 '존재하는 첫 번째'를 선택.
    """
    candidates = [
        "configs/rtmdet/rtmdet_l_8xb32-300e_coco.py",
        "configs/rtmdet/rtmdet_m_8xb32-300e_coco.py",
        "configs/rtmdet/rtmdet_s_8xb32-300e_coco.py",
        # 혹시 RTMDet가 없다면 2순위 후보
        "configs/yolox/yolox_l_8xb8-300e_coco.py",
        "configs/yolox/yolox_m_8xb8-300e_coco.py",
    ]

    for rel in candidates:
        p = MMD_ROOT / rel
        if p.exists():
            print("Selected base cfg:", p)
            return p

    # 마지막 안전장치: 그래도 없으면 에러로 명확히 알려줌
    raise FileNotFoundError(
        "RTMDet/YOLOX 후보 config를 찾지 못했습니다. "
        "MMD_ROOT/configs 아래 실제 파일명을 확인해 주세요."
    )


MMD_ROOT: /data/ephemeral/home/model/baseline/mmdetection
FULL_DATA_ROOT: /data/ephemeral/home/model/dataset
TRAIN_JSON_FULL: /data/ephemeral/home/model/dataset/train.json
TEST_JSON_FULL : /data/ephemeral/home/model/dataset/test.json
SAMPLE_SUB_DIR : /data/ephemeral/home/model/sample_submission
WORK_DIR_ROOT  : /data/ephemeral/home/model/work_dirs_single
